In [3]:
import cv2
from ultralytics import YOLO

# Instantiate the YOLO model
model = YOLO("yolov8s.pt")

# Initialize video capture from a video file
video_path = 'input_video.mp4'
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print(f"Error: Could not open video file: {video_path}")
    exit()

# 1. Initialize an empty set to store unique track IDs of detected people
unique_person_ids = set()

# Get the index for the 'person' class from the model.names dictionary
try:
    person_class_id = [key for key, val in model.names.items() if val == 'person'][0]
except IndexError:
    print(f"Hiba a person taggel")
    exit()

# Get video properties for output video writer
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize output video writer
output_video_path = 'output_video_with_detections.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for .mp4 file
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

if not out.isOpened():
    print(f"Error: Could not open video writer for {output_video_path}")
    exit()

# 2. Start a while loop that continues as long as cap.isOpened() is True
while cap.isOpened():
    # 3. Read a frame from the video capture object
    ret, frame = cap.read()

    # 4. Check if the frame was read successfully
    if not ret:
        print("End of video stream or an error occurred.")
        break

    # 5. Perform object detection and tracking on the current frame
    # Only track 'person' class with persist=True for continuous tracking
    results = model.track(frame, persist=True, classes=person_class_id, verbose=False)[0]

    # Initialize current frame's detected track IDs
    current_frame_person_ids = set()

    # 6. Iterate through the boxes attribute of the results object
    if results.boxes.id is not None:
        for i, box in enumerate(results.boxes):
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls_id = int(box.cls[0])
            conf = box.conf[0]
            track_id = int(results.boxes.id[i].item())

            # Ensure it's a 'person' detection
            if cls_id == person_class_id:
                current_frame_person_ids.add(track_id)
                unique_person_ids.add(track_id)

                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) # Green color

                # Add track ID and confidence score as text
                label = f"ID: {track_id} Conf: {conf:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Add total unique persons count to the frame
    total_unique_text = f"Total Unique Persons: {len(unique_person_ids)}"
    cv2.putText(frame, total_unique_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2) # Changed color to green

    # Write the annotated frame to the output video file
    out.write(frame)

# 8. Release the video capture object and the video writer object
cap.release()
out.release()
print(f"Total unique persons detected throughout the video: {len(unique_person_ids)}")
print(f"Annotated video saved to: {output_video_path}")

End of video stream or an error occurred.
Total unique persons detected throughout the video: 112
Annotated video saved to: output_video_with_detections.mp4


In [4]:
import cv2
from ultralytics import YOLO

# Instantiate the YOLO model
model = YOLO("yolov8s.pt")

# Initialize video capture from a video file
video_path = 'input_video.mp4'
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print(f"Error: Could not open video file: {video_path}")
    exit()

# 1. Initialize an empty set to store unique track IDs of detected people
unique_person_ids = set()

# Get the index for the 'person' class from the model.names dictionary
try:
    person_class_id = [key for key, val in model.names.items() if val == 'person'][0]
except IndexError:
    print(f"Hiba a person taggel")
    exit()

# Get video properties for output video writer
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize output video writer
output_video_path = 'output_video_with_detections_v2.mp4' # Changed output name to avoid overwriting
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for .mp4 file
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

if not out.isOpened():
    print(f"Error: Could not open video writer for {output_video_path}")
    exit()

# Initialize frame counters
processed_frames_count = 0
dropped_frames_count = 0

# 2. Start a while loop that continues as long as cap.isOpened() is True
while cap.isOpened():
    # 3. Read a frame from the video capture object
    ret, frame = cap.read()

    # 4. Check if the frame was read successfully
    if not ret:
        # If not successful, increment dropped frames and break if end of stream
        dropped_frames_count += 1
        print(f"End of video stream or an error occurred. Dropped frames: {dropped_frames_count}")
        break

    processed_frames_count += 1

    # 5. Perform object detection and tracking on the current frame
    # Only track 'person' class with persist=True for continuous tracking
    results = model.track(frame, persist=True, classes=person_class_id, verbose=False)[0]

    # Initialize current frame's detected track IDs
    current_frame_person_ids = set()

    # 6. Iterate through the boxes attribute of the results object
    if results.boxes.id is not None:
        for i, box in enumerate(results.boxes):
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls_id = int(box.cls[0])
            conf = box.conf[0]
            track_id = int(results.boxes.id[i].item())

            # Ensure it's a 'person' detection
            if cls_id == person_class_id:
                current_frame_person_ids.add(track_id)
                unique_person_ids.add(track_id)

                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) # Green color

                # Add track ID and confidence score as text
                label = f"ID: {track_id} Conf: {conf:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Add total unique persons count to the frame
    total_unique_text = f"Total Unique Persons: {len(unique_person_ids)}"
    cv2.putText(frame, total_unique_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Write the annotated frame to the output video file
    out.write(frame)

# 8. Release the video capture object and the video writer object
cap.release()
out.release()
print(f"Total frames processed: {processed_frames_count}")
print(f"Total frames dropped during reading: {dropped_frames_count}")
print(f"Total unique persons detected throughout the video: {len(unique_person_ids)}")
print(f"Annotated video saved to: {output_video_path}")

KeyboardInterrupt: 

In [ ]:
import requests

# Placeholder for a direct download URL of a stock street video
# You can find free stock videos on sites like Pexels, Pixabay, or Unsplash.
# Make sure to get a direct download link for an .mp4 file.
video_url = "YOUR_STOCK_VIDEO_DIRECT_DOWNLOAD_URL.mp4"
output_filename = "input_video.mp4"

print(f"Downloading video from {video_url}...")
try:
    response = requests.get(video_url, stream=True)
    response.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)

    with open(output_filename, 'wb') as video_file:
        for chunk in response.iter_content(chunk_size=8192):
            video_file.write(chunk)
    print(f"Video downloaded successfully as {output_filename}")
except requests.exceptions.RequestException as e:
    print(f"Error downloading video: {e}")
    print("Please ensure the URL is a direct link to an MP4 file and that you have an active internet connection.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")



In [1]:
!pip install opencv-python ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.2 MB/s eta 0:00:00


After downloading the video, you can run the previous code cell (`vVryKVdGCquH`) to process it.